# 01 - Feast Feature Store Setup

**Data Generation → Feast Apply → Materialize → Verify**

## Configuration Options

| Toggle | Operation | `false` (default) | `true` |
|--------|-----------|-------------------|--------|
| `use_ray` | Training `get_historical_features()` | File-based (single node) | KubeRay (distributed PIT joins) |
| `use_ray_dataprep` | DataPrep `feast materialize` | Single-node | Ray batch_engine (big data) |

### When to Enable Each

| Data Size | `use_ray` | `use_ray_dataprep` | Notes |
|-----------|-----------|-------------------|-------|
| **<1M rows** | `false` | `false` | Quick start, no Ray needed |
| **1-10M rows** | `true` | `false` | Ray for training PIT joins only |
| **>10M rows** | `true` | `true` | Full Ray: training + materialization |

```
┌─────────────────────────────────────────────────────────────┐
│                    FEAST CONFIGURATION                      │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  feature_store.yaml       (File offline + optional Ray)     │
│  ├─ feast apply           ✅ Always single-node             │
│  ├─ feast materialize     🔀 Ray batch_engine if enabled    │
│  └─ batch_engine: ray     → Only if use_ray_dataprep=true   │
│                                                             │
│  feature_store_ray.yaml   (Ray/KubeRay offline store)       │
│  └─ get_historical_features() → Only if use_ray=true        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```


In [ ]:
%pip install -q "feast[postgres]==0.59.0" pandas pyarrow psycopg2-binary pyyaml yamlmagic
%load_ext yamlmagic


In [ ]:
%%yaml config

# =============================================================================
# 🔀 MODE SELECTION - Set this based on your cluster and data size
# =============================================================================
# Training: get_historical_features() - point-in-time joins
use_ray: true           # true = KubeRay for distributed PIT joins, false = file-based

# DataPrep: feast materialize - sync offline → online store
use_ray_dataprep: false # true = Ray batch_engine for big data (>1M rows), false = single-node

# =============================================================================
# Paths (shared PVC)
# =============================================================================
feature_repo: /shared/feature_repo
data_dir: /shared/data

# =============================================================================
# Data Generation
# =============================================================================
data:
  start_date: "2022-01-01"
  weeks: 104          # 2 years of weekly data
  stores: 10
  departments: 5
  seed: 42

# =============================================================================
# Feast (PostgreSQL backend)
# =============================================================================
feast:
  project: sales_forecasting
  postgres_host: postgres.feast-trainer-demo.svc.cluster.local
  postgres_port: 5432
  postgres_db: feast
  postgres_user: feast
  postgres_password: feast123

# =============================================================================
# KubeRay (only used if use_ray: true)
# =============================================================================
kuberay:
  cluster_name: feast-ray
  namespace: feast-trainer-demo
  skip_tls: true


In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta, timezone

FEATURE_REPO = Path(config['feature_repo'])
DATA_DIR = Path(config['data_dir'])
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Feature Repo: {FEATURE_REPO}")
print(f"Data Dir: {DATA_DIR}")


## Generate Sales Data


In [ ]:
# Generate synthetic sales data with lag features
np.random.seed(config['data']['seed'])
base_date = datetime.fromisoformat(config['data']['start_date']).replace(tzinfo=timezone.utc)
weeks = config['data']['weeks']
stores = config['data']['stores']
depts = config['data']['departments']

print(f"Generating {weeks} weeks × {stores} stores × {depts} depts = {weeks*stores*depts:,} records...")

sales_records = []
for week in range(weeks):
    week_date = base_date + timedelta(weeks=week)
    week_of_year = week_date.isocalendar()[1]
    
    # Seasonality
    seasonal = 1 + 0.3 * np.sin(2 * np.pi * week_of_year / 52)
    is_holiday = 1 if week_of_year in [6, 36, 47, 51, 52] else 0  # Super Bowl, Labor Day, Thanksgiving, Christmas
    holiday_factor = 1.5 if is_holiday else 1.0
    
    for store_id in range(1, stores + 1):
        store_base = 15000 + store_id * 500
        for dept_id in range(1, depts + 1):
            dept_factor = 0.5 + (dept_id % 5) * 0.2
            weekly_sales = max(0, store_base * dept_factor * seasonal * holiday_factor + np.random.normal(0, 2000))
            
            sales_records.append({
                "store_id": store_id,
                "dept_id": dept_id,
                "event_timestamp": week_date,
                "weekly_sales": round(weekly_sales, 2),
                "is_holiday": is_holiday,
                "temperature": round(50 + 30 * np.sin(2 * np.pi * week_of_year / 52) + np.random.normal(0, 5), 1),
                "fuel_price": round(2.5 + 0.5 * np.sin(2 * np.pi * week / 52) + np.random.normal(0, 0.1), 2),
                "cpi": round(210 + week * 0.05 + np.random.normal(0, 1), 2),
                "unemployment": round(max(4, 8 - week * 0.02 + np.random.normal(0, 0.3)), 1),
            })

sales_df = pd.DataFrame(sales_records)
sales_df = sales_df.sort_values(["store_id", "dept_id", "event_timestamp"]).reset_index(drop=True)

# Add lag features
for lag in [1, 2, 4, 8, 52]:
    sales_df[f"lag_{lag}"] = sales_df.groupby(["store_id", "dept_id"])["weekly_sales"].shift(lag)

sales_df["rolling_mean_4w"] = sales_df.groupby(["store_id", "dept_id"])["weekly_sales"].transform(
    lambda x: x.rolling(4, min_periods=1).mean()
)

# Drop rows with NaN lags
sales_df = sales_df.dropna()
print(f"✅ Sales features: {len(sales_df):,} rows")
sales_df.head()


In [ ]:
# Generate store metadata
store_records = []
for store_id in range(1, stores + 1):
    store_type = ["A", "B", "C"][store_id % 3]
    store_size = {"A": 200000, "B": 150000, "C": 100000}[store_type]
    region = ["West", "South", "Midwest", "Northeast"][store_id % 4]
    
    for dept_id in range(1, depts + 1):
        store_records.append({
            "store_id": store_id,
            "dept_id": dept_id,
            "event_timestamp": base_date,
            "store_type": store_type,
            "store_size": store_size,
            "region": region,
        })

store_df = pd.DataFrame(store_records)
print(f"✅ Store features: {len(store_df):,} rows")

# Save parquet files
sales_df.to_parquet(DATA_DIR / "sales_features.parquet", index=False)
store_df.to_parquet(DATA_DIR / "store_features.parquet", index=False)
print(f"\n✅ Saved to {DATA_DIR}:")
!ls -lh {DATA_DIR}/*.parquet


## Create Feast Configs

Generate **two** configs:
- `feature_store.yaml` - File-based (for apply/materialize)
- `feature_store_ray.yaml` - KubeRay (for training)


In [ ]:
import shutil
import yaml

USE_RAY = config.get('use_ray', False)
USE_RAY_DATAPREP = config.get('use_ray_dataprep', False)
FEATURE_REPO.mkdir(parents=True, exist_ok=True)

print("=" * 60)
print("🔀 Configuration:")
print(f"   Training (get_historical_features): {'KubeRay' if USE_RAY else 'File-based'}")
print(f"   DataPrep (materialize):             {'Ray batch_engine' if USE_RAY_DATAPREP else 'Single-node'}")
print("=" * 60)

# Source configs (version-controlled in repo)
REPO_CONFIG_DIR = Path("../feature_repo")

# Copy static configs from repo to PVC
shutil.copy(REPO_CONFIG_DIR / "feature_store.yaml", FEATURE_REPO / "feature_store.yaml")
shutil.copy(REPO_CONFIG_DIR / "features.py", FEATURE_REPO / "features.py")
print(f"✅ Copied feature_store.yaml and features.py from {REPO_CONFIG_DIR}")

# Add Ray batch_engine for big data (if enabled)
if USE_RAY_DATAPREP:
    with open(FEATURE_REPO / "feature_store.yaml") as f:
        fs_config = yaml.safe_load(f)
    
    kr = config['kuberay']
    fs_config["batch_engine"] = {
        "type": "ray.engine",
        "ray_address": f"ray://feast-ray-head-svc.{kr['namespace']}.svc.cluster.local:10001",
    }
    
    with open(FEATURE_REPO / "feature_store.yaml", "w") as f:
        f.write("# File-based offline store + Ray batch_engine (big data)\n")
        yaml.dump(fs_config, f, default_flow_style=False)
    print("   + Added batch_engine: ray.engine (for distributed materialize)")

# Copy Ray config for training (if enabled)
if USE_RAY:
    shutil.copy(REPO_CONFIG_DIR / "feature_store_ray.yaml", FEATURE_REPO / "feature_store_ray.yaml")
    print(f"✅ Copied feature_store_ray.yaml (KubeRay for training)")
else:
    ray_yaml = FEATURE_REPO / "feature_store_ray.yaml"
    if ray_yaml.exists():
        ray_yaml.unlink()
    print("ℹ️  Skipped feature_store_ray.yaml (use_ray=false)")

print()
!ls -la {FEATURE_REPO}/


## Feast Apply

Register feature definitions to PostgreSQL registry.


In [ ]:
%%bash -s "$FEATURE_REPO"
cd $1 && feast apply


## Feast Materialize

Populate online store (PostgreSQL) for low-latency inference.


In [ ]:
# Materialize: offline store → online store
start_ts = config['data']['start_date'] + "T00:00:00"
end_ts = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S")

!cd {FEATURE_REPO} && feast materialize {start_ts} {end_ts}


## Verify Features


In [ ]:
from feast import FeatureStore

store = FeatureStore(repo_path=str(FEATURE_REPO))
print(f"Project: {store.project}")
print(f"\nFeature Views:")
for fv in store.list_feature_views():
    print(f"  - {fv.name}: {len(fv.features)} features, ttl={fv.ttl}")
print(f"\nFeature Services:")
for fs in store.list_feature_services():
    print(f"  - {fs.name}: {fs.description}")


In [ ]:
# Test online feature retrieval (low-latency)
import time
entity_rows = [{"store_id": 1, "dept_id": 1}, {"store_id": 5, "dept_id": 3}]

t0 = time.time()
online_features = store.get_online_features(
    features=[
        "sales_features:weekly_sales",
        "sales_features:lag_1",
        "sales_features:rolling_mean_4w",
        "store_features:store_type",
        "store_features:store_size",
    ],
    entity_rows=entity_rows,
).to_dict()
latency_ms = (time.time() - t0) * 1000

print(f"✅ Online features retrieved in {latency_ms:.1f}ms")
pd.DataFrame(online_features)


## Summary

**What we created:**

| Component | Location | Purpose |
|-----------|----------|---------|
| `sales_features.parquet` | `/shared/data/` | Weekly sales + lag features |
| `store_features.parquet` | `/shared/data/` | Store metadata |
| `feature_store.yaml` | `/shared/feature_repo/` | File-based (apply/materialize) |
| `feature_store_ray.yaml` | `/shared/feature_repo/` | KubeRay (training) |

**Next:** Run `02-training.ipynb` to train with `get_historical_features()` via KubeRay.
